# Django ORM - Praca z danymi (komendy DML i DQL)

### Co potrzebujemy do uruchomienia tego notebooka?
* jupyter
* django-extensions

<code>$ pip install jupyter django-extensions</code>

Więcej informacji znajdziesz [tutaj](https://medium.com/ayuth/how-to-use-django-in-jupyter-notebook-561ea2401852)

## Definicja modelu

<code>class Task(models.Model):
    name = models.CharField(max_length=64)
</code>
<code>
    def __str__(self):
        return f"{self.name}"
</code>

W dalszej części notebooka przyjęto założenie, że powyższy model został stworzony w aplikacji taskapp w pliku models.py oraz, że przygotowowano i zrealizowano migracje dla tego modelu.

In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from form_app5.models import Task

## C - CREATE (CRUD) - DML (Data Manipulation Language)

### Klauzula INSERT

In [5]:
# Metoda I - metoda create menadżera modelu (objects)

res = Task.objects.create(name="Smażenie")

In [3]:
# Metoda II - metoda save instancji modelu

task = Task()
task.name = "Pieczenie"
task.save()

In [4]:
# W drugiej metodzie wartości parametrów można oczywiście przekazać w inicjalizatorze.

task = Task(name="Gotowanie")
task.save()

In [6]:
# Przed przejściem do litery R dodajmy jeszcze kilka wpisów do tabeli, żeby mieć 
# co analizować podczas poznawania metod implementujących instrukcje z rodziny READ.

Task.objects.create(name="Szukanie")
Task.objects.create(name="Szukanie")
Task.objects.create(name="Programowanie")
Task.objects.create(name="Pływanie")
Task.objects.create(name="Pranie")
Task.objects.create(name="Dodawanie")

<Task: Dodawanie>

## R - Read (CRUD) - DQL (Data Query Language)

### Klauzula SELECT

Operacja READ w SQL to instrukcja SELECT z całą swoją rozbudowaną składnią i operatorami takimi jak: LIKE, GROUP_BY, ORDER_BY, HAVING, IN, JOIN, UNION, ...
Menadżer modelu (**.objects**) posiada odpowiednie metody implementujące część tych instrukcji.

In [7]:
print(dir(Task.objects))

['__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_constructor_args', '_db', '_get_queryset_methods', '_hints', '_insert', '_queryset_class', '_set_creation_counter', '_update', 'aaggregate', 'abulk_create', 'abulk_update', 'acontains', 'acount', 'acreate', 'aearliest', 'aexists', 'aexplain', 'afirst', 'aget', 'aget_or_create', 'aggregate', 'ain_bulk', 'aiterator', 'alast', 'alatest', 'alias', 'all', 'annotate', 'aupdate', 'aupdate_or_create', 'auto_created', 'bulk_create', 'bulk_update', 'check', 'complex_filter', 'contains', 'contribute_to_class', 'count', 'create', 'creation_counter', 'dates', 'datetimes', 'db', 'db_manager', 'deconstruct'

Metoda all menadżera modelu odpowiada instrukcji SELECT *

In [10]:
tasks = Task.objects.all()

print(tasks)

<QuerySet [<Task: Mycie>, <Task: Pieczenie>, <Task: Gotowanie>, <Task: Smażenie>, <Task: Szukanie>, <Task: Szukanie>, <Task: Programowanie>, <Task: Pływanie>, <Task: Pranie>, <Task: Dodawanie>]>


Metoda all zwraca obiekt klasy QuerySet. Jest to klasa reprezentująca zapytanie do bazy. Obiekt ten istnieje przed wykonaniem zapytania, a po wykonaniu zapytania jest uzupełniana o odpowiedź z bazy. Wśród swoich pól klasa QuerySet posiada atrybut query przechowujący zapytanie sql, które zostanie (lub już zostało) wykonane na bazie.

In [11]:
print(tasks.query)

SELECT "form_app5_task"."id", "form_app5_task"."name" FROM "form_app5_task"


Wśród wielu metod udostępnianych przez QuerySet można znaleźć takie, które widzieliśmy już wśród metod udostępnianych przez menadżera modelu, np. all, filter, exclude, union, get, first, last, ...

In [12]:
print(dir(tasks))

['__aiter__', '__and__', '__bool__', '__class__', '__class_getitem__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '__xor__', '_add_hints', '_annotate', '_batched_insert', '_chain', '_check_bulk_create_options', '_check_operator_queryset', '_check_ordering_first_last_queryset_aggregation', '_clone', '_combinator_query', '_db', '_defer_next_filter', '_deferred_filter', '_earliest', '_extract_model_params', '_fetch_all', '_fields', '_filter_or_exclude', '_filter_or_exclude_inplace', '_for_write', '_has_filters', '_hints', '_insert', '_iterable_class', '_iterator', '_known_related_objects', '_merge_known_related_

Wynika to z faktu, że każdy menadżer modelu posiada swój własny, początkowy QuerySet. Kiedy wywołujemy na menadżerze modelu metodę, która znajduje się w interfejsie klasy QuerySet (np. all), to menadżer modelu po prostu wywołuje tą metodę na swoim własnym QuerySet-cie (oddelegowuje wykonanie metody do swojego QuerySet-a).

Czyli metoda all działa na obiekcie QuerySet (początkowym QuerySet-cie menadżera modelu) i zwraca obiekt klasy QuerySet. Skoro to co zwraca metoda all to obiekt klasy QuerySet, to ten obiekt posiada takie metody jak all, filter, ... Wynika z tego, że metodę all możemy łańcuchować, tzn. wywoływać jedną po drugiej.

In [13]:
tasks = Task.objects.all().all().all()
print(tasks)

<QuerySet [<Task: Mycie>, <Task: Pieczenie>, <Task: Gotowanie>, <Task: Smażenie>, <Task: Szukanie>, <Task: Szukanie>, <Task: Programowanie>, <Task: Pływanie>, <Task: Pranie>, <Task: Dodawanie>]>


QuerySet posiada więcej metod, które w wyniku działania zwracają obiekty klasy QuerySet (np. filter, exclude, order_by, ...) i o ile łańcuchownie samej metody all nie ma za bardzo sensu, tak załańcuchowanie np. metody order_by po wywołaniu metody all w celu posortowania wpisów już ma.

In [14]:
tasks = Task.objects.all().order_by('name')
print(tasks.query)

SELECT "form_app5_task"."id", "form_app5_task"."name" FROM "form_app5_task" ORDER BY "form_app5_task"."name" ASC


In [15]:
print(tasks)  # konsumowanie queryseta

<QuerySet [<Task: Dodawanie>, <Task: Gotowanie>, <Task: Mycie>, <Task: Pieczenie>, <Task: Pranie>, <Task: Programowanie>, <Task: Pływanie>, <Task: Smażenie>, <Task: Szukanie>, <Task: Szukanie>]>


Struktura umożliwiająca łańcuchowanie metod na tyle często pojawia się w programowaniu, że posiada nawet swoją nazwę. Mówimy, że QuerySet implementuje wzorzec fluent interface (płynny interfejs).

Ale nie wszystkie metody QuerySet (i menadżera modelu) zwracają QuerySet. Na przykład metody first i last zwracają odpowiednio pierwszy i ostatni element QuerySeta (czyli instancje modelu). Takie metody nie zwracają obiektu klasy QuerySet (nie implementują wzorca fluent interface) i dlatego po ich użyciu nie można już użyć żadnej innej metody obiektu QuerySet do łańcuchowania.

In [18]:
task = Task.objects.first()
print(task)
print(type(task))

Mycie
<class 'form_app5.models.Task'>


In [19]:
task = Task.objects.last()
print(task)

Dodawanie


### Dostęp do wartości w poszczególnych kolumnach wpisu

Do wartości w poszczególnych kolumnach wpisu dostajemy się poprzez notacją obiektową (odwołujemy się do atrybutu instancji modelu). Jaką wartość w kolumnie name ma ostatni wpis z tabelki Task?

In [20]:
print(task.name)

Dodawanie


### Filtry - metody filter i get (klauzula WHERE)

#### Metoda I - filter

Metoda filter menadżera modelu (i Queryset-a) odpowiada klauzuli WHERE.

In [23]:
task = Task.objects.filter(name="Szukanie")
print(task)

<QuerySet [<Task: Szukanie>, <Task: Szukanie>]>


Widzimy, że metoda filter zwraca obiekt QuerySet. QuerySet może być pusty.

In [24]:
task = Task.objects.filter(name="Coś czego nie ma w tabeli")
print(task)

<QuerySet []>


QuerySet może być jednoelementowy.

In [25]:
task = Task.objects.filter(name="Programowanie")
print(task)

<QuerySet [<Task: Programowanie>]>


#### Metoda II - get

W odróżnieniu od metody filter, metoda get zwraca instancję modelu (a nie obiekt klasy QuerySet).

In [26]:
task = Task.objects.get(name="Programowanie")
print(task)
print(type(task))

Programowanie
<class 'form_app5.models.Task'>


Metoda get oczekuje, że w wyniku otrzyma **jeden i tylko jeden** wpis.

Jeżeli zapytanie nie zwróci żadnego wpisu metoda get rzuci wyjątek *DoesNotExist*.

In [27]:
task = Task.objects.get(name="Coś czego nie ma w tabeli")

DoesNotExist: Task matching query does not exist.

Jeżeli zapytanie zwróci więcej niż jeden wpis metoda get rzuci wyjątek *MultipleObjectsReturned*.

In [28]:
Task.objects.get(name="Szukanie")

MultipleObjectsReturned: get() returned more than one Task -- it returned 2!

Podsumowując, metoda get w odróżnieniu od metody filter:
* zwraca instancję modelu (a nie obiekty klasy QuerySet)
* jeżeli w wyniku filtrowania otrzymamy pustą odpowiedź rzuci wyjątek *DoesNotExist* (a nie zwróci pusty QuerySet)
* jeżeli w wyniku filtrowania otrzymamy więcej niż jeden wpis rzuci wyjątek *MultipleObjectsReturned* (a nie zwróci wieloelementowy QuerySet)

#### Field lookups (operatory klauzuli WHERE)

Klauzula WHERE posiada wiele operatorów takich jak: LIKE, IN, >, <, ... We frameworku Django implementujemy te operatory za pomocą tak zwanych field lookups. Składniowo używanie lookupów polega na dodaniu po nazwie kolumny w filtrze dwóch znaków __ a następnie odpowiedniego słówka, np. contains, startswith, lte (less than or equal), gt (greater than) ...

Znajdźmy wszystkie wpisy z tabeli Task, dla których wartość w kolumnie name rozpoczyna się na Pr.

In [29]:
tasks = Task.objects.filter(name__startswith="Pr")
print(tasks)

<QuerySet [<Task: Programowanie>, <Task: Pranie>]>


Popatrzmy na sql

In [30]:
print(tasks.query)

SELECT "form_app5_task"."id", "form_app5_task"."name" FROM "form_app5_task" WHERE "form_app5_task"."name" LIKE Pr% ESCAPE '\'


Znajdźmy wszystkie wpisy z tabli Task, dla których wartość w kolumnie id jest większa bądź równa 5.

In [31]:
tasks = Task.objects.filter(id__gte=5)
print(tasks)

<QuerySet [<Task: Szukanie>, <Task: Szukanie>, <Task: Programowanie>, <Task: Pływanie>, <Task: Pranie>, <Task: Dodawanie>]>


In [32]:
# sql?
print(tasks.query)

SELECT "form_app5_task"."id", "form_app5_task"."name" FROM "form_app5_task" WHERE "form_app5_task"."id" >= 5


Znajdźmy wszystkie wpisy z tabeli Task, dla których wartość w kolumnie name zawiera ow.

In [34]:
tasks = Task.objects.filter(name__contains="ow")
print(tasks)

<QuerySet [<Task: Gotowanie>, <Task: Programowanie>]>
SELECT "form_app5_task"."id", "form_app5_task"."name" FROM "form_app5_task" WHERE "form_app5_task"."name" LIKE %ow% ESCAPE '\'


### Indeksowanie, wycinki (operatory LIMIT i OFFSET)

Klasa QuerySet wspiera indeksowanie oraz wycinki

Znajdźmy czwarty wpis w tabeli Task (indeksowanie od 0).

In [35]:
task = Task.objects.all()[3]
print(task)
print(type(task))  # instancja modelu

Smażenie
<class 'form_app5.models.Task'>


Znajdźmy pięc pierwszych wpisów w tabeli Task.

In [36]:
tasks = Task.objects.all()[:5]
print(tasks)

<QuerySet [<Task: Mycie>, <Task: Pieczenie>, <Task: Gotowanie>, <Task: Smażenie>, <Task: Szukanie>]>


In [37]:
# sql?
print(tasks.query)

SELECT "form_app5_task"."id", "form_app5_task"."name" FROM "form_app5_task" LIMIT 5


Znajdźmy wszystkie wpisy w tabeli Task poza pięcioma pierwszymi.

In [38]:
tasks = Task.objects.all()[5:]
print(tasks)

<QuerySet [<Task: Szukanie>, <Task: Programowanie>, <Task: Pływanie>, <Task: Pranie>, <Task: Dodawanie>]>


In [39]:
# sql?
print(tasks.query)

SELECT "form_app5_task"."id", "form_app5_task"."name" FROM "form_app5_task" LIMIT -1 OFFSET 5


Znajdźmy co drugi wpis z tabeli Task (wycinki)

In [40]:
tasks = Task.objects.all()[::2]
print(tasks)
print(type(tasks))

[<Task: Mycie>, <Task: Gotowanie>, <Task: Szukanie>, <Task: Programowanie>, <Task: Pranie>]
<class 'list'>


**Uwaga!**

W ostatnim przykładzie wycinek zwraca **listę instancji modelu**, a nie QuerySet.

**Uwaga!** 

QuerySet w odróżnieniu od listy Pythonowej nie obsługuje negatywnych indeksów.